In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# Using a pre-trained convnet

A common and highly effective approach to deep learning on small image datasets is to leverage a pre-trained network. A pre-trained network 
is simply a saved network previously trained on a large dataset, typically on a large-scale image classification task. If this original 
dataset is large enough and general enough, then the spatial feature hierarchy learned by the pre-trained network can effectively act as a 
generic model of our visual world, and hence its features can prove useful for many different computer vision problems, even though these 
new problems might involve completely different classes from those of the original task. For instance, one might train a network on 
ImageNet (where classes are mostly animals and everyday objects) and then re-purpose this trained network for something as remote as 
identifying furniture items in images. Such portability of learned features across different problems is a key advantage of deep learning 
compared to many older shallow learning approaches, and it makes deep learning very effective for small-data problems.

In our case, we will consider a large convnet trained on the ImageNet dataset (1.4 million labeled images and 1000 different classes). 

La idea principal es utilitzar diferents arquitectures (VGG16, VGG19, ResNet, Inception, Inception-ResNet, Xception) para despues generar un mix que nos permita obtener mejores resultados.

There are two ways to leverage a pre-trained network: *feature extraction* and *fine-tuning*. Fine-tuning is much slower and more expensive, but allows 
us to leverage data augmentation during training: extending the `conv_base` model and running it end-to-end on the inputs. Note that this 
technique is in fact so expensive that you should only attempt it if you have access to a GPU: it is absolutely intractable on CPU. If you 
cannot run your code on GPU, then feature extraction is the way to go.

Let's start.

## Fine-tuning

Fine-tuning consists in unfreezing a few of the top layers 
of a frozen model base used for feature extraction, and jointly training both the newly added part of the model (in our case, the 
fully-connected classifier) and these top layers. This is called "fine-tuning" because it slightly adjusts the more abstract 
representations of the model being reused, in order to make them more relevant for the problem at hand.

![fine-tuning VGG16](https://s3.amazonaws.com/book.keras.io/img/ch5/vgg16_fine_tuning.png)

"Freezing" a layer or set of layers means preventing their weights from getting updated during training. If we don't do this, then the representations that were previously learned by the convolutional base would get modified during training. Since the `Dense` layers on top are randomly initialized, very large weight updates would be propagated through the network, effectively destroying the representations previously learned.

In Keras, freezing a network is done by setting its `trainable` attribute to `False`.    

Thus the steps for fine-tuning a network are as follow:

* 1) Add your custom network on top of an already trained base network.
* 2) Freeze the base network.
* 3) Train the part you added.
* 4) Unfreeze some layers in the base network.
* 5) Jointly train both these layers and the part you added.

## Part 1 - Add custom network on top of pretrained network

La primera parte se corresponde on los puntos 1, 2 y 3:

Let's instantiate the VGG16 model:

In [2]:
from keras.applications import ResNet50

conv_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(197, 197, 3))

We passed three arguments to the constructor:

* `weights`, to specify which weight checkpoint to initialize the model from
* `include_top`, which refers to including or not the densely-connected classifier on top of the network. By default, this 
densely-connected classifier would correspond to the 1000 classes from ImageNet. Since we intend to use our own densely-connected 
classifier (with only two classes, cat and dog), we don't need to include it.
* `input_shape`, the shape of the image tensors that we will feed to the network. This argument is purely optional: if we don't pass it, 
then the network will be able to process inputs of any size.

Here's the detail of the architecture of the VGG16 convolutional base: it's very similar to the simple convnets that you are already 
familiar with.

In [3]:
conv_base.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 197, 197, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 99, 99, 64)    9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 99, 99, 64)    256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 99, 99, 64)    0           bn_conv1[0][0]                   
___________________________________________________________________________________________

The final feature map has shape `(4, 4, 512)`. That's the feature on top of which we will stick a densely-connected classifier.

Because models behave just like layers, you can add a model (like our `conv_base`) to a `Sequential` model just like you would add a layer. 
So you can do the following:

In [4]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='sigmoid'))

This is what our model looks like now:

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 24,114,826
Trainable params: 24,061,706
Non-trainable params: 53,120
_________________________________________________________________


As you can see, the convolutional base of VGG16 has 14,714,688 parameters, which is very large. The classifier we are adding on top has 2 
million parameters.

Before we compile and train our model, a very important thing to do is to freeze the convolutional base. 

In [6]:
conv_base.trainable = False

With this setup, only the weights from the two `Dense` layers that we added will be trained. That's a total of four weight tensors: two per 
layer (the main weight matrix and the bias vector). Note that in order for these changes to take effect, we must first compile the model. 
If you ever modify weight trainability after compilation, you should then re-compile the model, or these changes would be ignored.

Now we can start training our model, with the same data augmentation configuration that we used in our previous example:

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

base_dir = './logos/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')
test_dir = os.path.join(base_dir, 'test')

print (train_dir)

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20



train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=50,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(197, 197),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(197, 197),
        batch_size=20,
        class_mode='categorical')

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=200,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)

./logos/train
Found 434 images belonging to 10 classes.
Found 140 images belonging to 10 classes.
Epoch 1/30
200/200 [==============================] - 144s - loss: 0.9377 - acc: 0.7806 - val_loss: 2.2411 - val_acc: 0.1270
Epoch 2/30
200/200 [==============================] - 140s - loss: 0.0785 - acc: 0.9910 - val_loss: 2.1770 - val_acc: 0.2950
Epoch 3/30
200/200 [==============================] - 139s - loss: 0.0174 - acc: 0.9975 - val_loss: 1.7328 - val_acc: 0.5260
Epoch 4/30
200/200 [==============================] - 139s - loss: 0.0083 - acc: 0.9982 - val_loss: 1.2056 - val_acc: 0.6970
Epoch 5/30
200/200 [==============================] - 139s - loss: 0.0052 - acc: 0.9985 - val_loss: 0.9997 - val_acc: 0.7680
Epoch 6/30
200/200 [==============================] - 139s - loss: 0.0030 - acc: 0.9991 - val_loss: 0.9742 - val_acc: 0.7580
Epoch 7/30
200/200 [==============================] - 139s - loss: 0.0015 - acc: 0.9997 - val_loss: 0.9985 - val_acc: 0.7550
Epoch 8/30
200/200 [=======

In [8]:
model.save('model_resnet50_pre_train.h5')

Let's plot our results again:

In [9]:
import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

As you can see, we reach a validation accuracy of about 96%. This is much better than our small convnet trained from scratch.

## Part 2 - Train last layer of pretrained network

We have already completed the first 3 steps when doing feature extraction. Let's proceed with the 4th step: we will unfreeze our `conv_base`, 
and then freeze individual layers inside of it.

As a reminder, this is what our convolutional base looks like:

In [16]:
conv_base.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 197, 197, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 99, 99, 64)    9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 99, 99, 64)    256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 99, 99, 64)    0           bn_conv1[0][0]                   
___________________________________________________________________________________________


We will fine-tune the last 3 convolutional layers, which means that all layers up until `block4_pool` should be frozen, and the layers 
`block5_conv1`, `block5_conv2` and `block5_conv3` should be trainable.

Why not fine-tune more layers? Why not fine-tune the entire convolutional base? We could. However, we need to consider that:

* Earlier layers in the convolutional base encode more generic, reusable features, while layers higher up encode more specialized features. It is 
more useful to fine-tune the more specialized features, as these are the ones that need to be repurposed on our new problem. There would 
be fast-decreasing returns in fine-tuning lower layers.
* The more parameters we are training, the more we are at risk of overfitting. The convolutional base has 15M parameters, so it would be 
risky to attempt to train it on our small dataset.

Thus, in our situation, it is a good strategy to only fine-tune the top 2 to 3 layers in the convolutional base.

Let's set this up, starting from where we left off in the previous example:

In [17]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'res5c_branch2a':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [18]:
conv_base.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 197, 197, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 99, 99, 64)    9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 99, 99, 64)    256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 99, 99, 64)    0           bn_conv1[0][0]                   
___________________________________________________________________________________________

Now we can start fine-tuning our network. We will do this with the RMSprop optimizer, using a very low learning rate. The reason for using 
a low learning rate is that we want to limit the magnitude of the modifications we make to the representations of the 3 layers that we are 
fine-tuning. Updates that are too large may harm these representations.

Now let's proceed with fine-tuning:

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=200,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/15
200/200 [==============================] - 58s - loss: 0.0023 - acc: 0.9993 - val_loss: 0.3023 - val_acc: 0.9583
Epoch 2/15
200/200 [==============================] - 56s - loss: 1.6554e-04 - acc: 1.0000 - val_loss: 0.2526 - val_acc: 0.9623
Epoch 3/15
200/200 [==============================] - 56s - loss: 2.5951e-04 - acc: 0.9999 - val_loss: 0.2293 - val_acc: 0.9648
Epoch 4/15
200/200 [==============================] - 56s - loss: 1.2387e-04 - acc: 1.0000 - val_loss: 0.2321 - val_acc: 0.9652
Epoch 5/15
200/200 [==============================] - 56s - loss: 1.7979e-04 - acc: 1.0000 - val_loss: 0.2372 - val_acc: 0.9634
Epoch 6/15
200/200 [==============================] - 56s - loss: 1.8910e-04 - acc: 1.0000 - val_loss: 0.2313 - val_acc: 0.9631
Epoch 7/15
200/200 [==============================] - 56s - loss: 5.1837e-05 - acc: 1.0000 - val_loss: 0.2412 - val_acc: 0.9630
Epoch 8/15
200/200 [==============================] - 56s - loss: 4.6481e-05 - acc: 1.0000 - val_loss: 0.243

In [20]:
model.save('model_resnet50.h5')

In [21]:
def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

plt.plot(epochs,
         smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
         smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

We are seeing a nice 1% absolute improvement.

Note that the loss curve does not show any real improvement (in fact, it is deteriorating). You may wonder, how could accuracy improve if the 
loss isn't decreasing? The answer is simple: what we display is an average of pointwise loss values, but what actually matters for accuracy 
is the distribution of the loss values, not their average, since accuracy is the result of a binary thresholding of the class probability 
predicted by the model. The model may still be improving even if this isn't reflected in the average loss.

We can now finally evaluate this model on the test data:

In [15]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

Found 140 images belonging to 10 classes.


ValueError: Error when checking input: expected input_1 to have shape (None, 197, 197, 3) but got array with shape (20, 150, 150, 3)

Here we get a test accuracy of 97%. In the original Kaggle competition around this dataset, this would have been one of the top results. 
However, using modern deep learning techniques, we managed to reach this result using only a very small fraction of the training data 
available (about 10%). There is a huge difference between being able to train on 20,000 samples compared to 2,000 samples!

## Take-aways: using convnets with small datasets

Here's what you should take away from the exercises of these past two sections:

* Convnets are the best type of machine learning models for computer vision tasks. It is possible to train one from scratch even on a very 
small dataset, with decent results.
* On a small dataset, overfitting will be the main issue. Data augmentation is a powerful way to fight overfitting when working with image 
data.
* It is easy to reuse an existing convnet on a new dataset, via feature extraction. This is a very valuable technique for working with 
small image datasets.
* As a complement to feature extraction, one may use fine-tuning, which adapts to a new problem some of the representations previously 
learned by an existing model. This pushes performance a bit further.

Now you have a solid set of tools for dealing with image classification problems, in particular with small datasets.